In [1]:
import os,sys
import pandas as pd

In [3]:

sample_info = pd.DataFrame(columns=["sample_name", "trimed_forward_fq"])


data_path = os.path.abspath("../data") + "/"
trimed_path = os.path.abspath("../trimmed/") + "/"
for path in os.listdir(data_path):
    # print(os.path.abspath(path))
    if os.path.isdir(data_path + path):
        sample_name = path
        trimed_forward_fq = trimed_path + sample_name +"_trimed.fastq.gz"
        sample_info = pd.concat([sample_info, pd.DataFrame([[sample_name, trimed_forward_fq]], columns=["sample_name", "trimed_forward_fq"])])

sample_info = sample_info.reset_index(drop=True)

In [5]:
# 去掉 公共数据
public_sample_info = sample_info.loc[ sample_info["sample_name"].str.contains("SRR"), :]
sample_info = sample_info.loc[~sample_info["sample_name"].str.contains("SRR"), :]

## bowtie2 比对

* 目前来看，后续想用 RiboCode call ORF ，必须用 STAR 进行比对

In [15]:
refernce_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/ncbi_assembly/bowtie2_index/GCA_000001405.29_GRCh38.p14_genomic"

# 根据要求之分析两个样本
sample_info_select = sample_info.loc[sample_info["sample_name"].str.contains("RP") | sample_info["sample_name"].str.contains("siNC-1-T") , :]
# sample_info_select.iloc[0,] = [sample_info_select.iloc[0,]["sample_name"], "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R1_001.fastq.gz", "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R2_001.fastq.gz"]

def rum_sbatch(x):
    # 线程数在这里修改
    command = f'sbatch step2_mapping_single.sh -i {x["trimed_forward_fq"]}  -t 12 -n {x["sample_name"]} -r {refernce_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info.apply(rum_sbatch, axis=1)

process sample :  SRR1630831
Submitted batch job 26518976


0    None
dtype: object

### 换成 genecode 的 index 再试一下

In [7]:
sample_info["sample_name"] = sample_info["sample_name"] + "_genecode"

In [10]:
refernce_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/bowtie2_index/GRCh38.p13.genome"


# sample_info_select.iloc[0,] = [sample_info_select.iloc[0,]["sample_name"], "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R1_001.fastq.gz", "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/learn/riboseq/wh_data/smmc-siYB1-2-RP/smmc-siYB1-2-RP_S15_L001_R2_001.fastq.gz"]

def rum_sbatch(x):
    # 线程数在这里修改
    command = f'sbatch step2_mapping_single.sh -i {x["trimed_forward_fq"]}  -t 12 -n {x["sample_name"]} -r {refernce_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info.apply(rum_sbatch, axis=1)

process sample :  SRR1630831_genecode
Submitted batch job 26518784


0    None
dtype: object

## STAR 比对

* 目前来看，后续想用 RiboCode call ORF ，必须用 STAR 进行比对

In [32]:
reference_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/STAR_index/"

sample_info_star = sample_info.copy()
sample_info_star["sample_name"] = sample_info_star["sample_name"] + "_star"
sample_info_star.iloc[0,1] = sample_info_star.iloc[0,1].replace(".gz", "")

def rum_sbatch(x):
    # 线程数在这里修改
    thread = 16
    command = f'sbatch step2_mapping_single_star.sh -i {x["trimed_forward_fq"]}  -t {thread} -n {x["sample_name"]} -r {reference_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

sample_info_star.apply(rum_sbatch, axis=1)

process sample :  SRR1630831_star
Submitted batch job 26521382


0    None
dtype: object

In [6]:
reference_path = "/dssg/home/acct-bioxsyy/bioxsyy-xyj/xyj/reference/human/GRCh38/genecode_assembly/STAR_index/"

public_sample_info_star = public_sample_info.copy()
public_sample_info_star["sample_name"] = public_sample_info_star["sample_name"] + "_star"

def rum_sbatch(x):
    # 线程数在这里修改
    thread = 16
    command = f'sbatch step2_mapping_star.sh -i {x["trimed_forward_fq"]}  -t {thread} -n {x["sample_name"]} -r {reference_path} '
    print(f'process sample :  {x["sample_name"]}')
    os.system(command)

public_sample_info_star.apply(rum_sbatch, axis=1)

process sample :  SRR1630831_star
Submitted batch job 26529305


1    None
dtype: object